<a href="https://colab.research.google.com/github/mld3/hack_aotearoa_intro_ml/blob/master/colab_worksheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started
Check scikit-learn version, install tqdm if necessary, load useful packages.

In [0]:
import sklearn
import pandas as pd
import numpy as np
import itertools
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
with open("/content/gdrive/My Drive/Hack_Aotearoa/helper_colab.py") as f:
    with open("helper_colab.py", "w") as f1:
        for line in f:
            f1.write(line)

In [0]:
from helper_colab import *

# 1 - Feature extraction

### 1a)
Study generate feature vector(df) in the helper.py file. The input to this function is a pd.DataFrame containing the data for a single patient admission. The output is a python dictio- nary object, corresponding to a d-dimensional feature vector for this patient (with missing values). The keys of this dictionary are feature names, and the values are the corresponding feature values. For the time-invariant variables, we use the raw values. We replace unknown observations (−1) with unde- fined (use np.nan), and name these features with the original variable names. For each time-varying variable, we compute the mean of all measurements for that variable.

(i) Read the [documentation](https://physionet.org/challenge/2012/\#general-descriptors) on the variable ICUType, and reflect on the current feature representation of this variable.**What does such a representation imply, when using a linear classifier? How else might you represent this variable (as possibly more than one feature)?**

(ii) Here we only consider the mean of the numerical variables. **What limitations are associated with this representation? What other summary statistics could be useful?**



### 1b)
Study impute missing values(X). Given a feature matrix X (where each row corresponds to a patient admission and each column a feature) with missing values, we consider each feature column independently. For each column, we impute the missing values by replacing it with the mean value of the observed values in that column.**What assumptions does this imputation approach make? How else might you handle missing data?**

### 1c)
Notice that many of these feature values lie on very different scales. To address this, we normalize each feature column xd to have range between 0 and 1. **Why might it be useful to scale the features in this way?**

### 1d)
Review the implementation of get train test split(). This helper function uses the three functions described above. First, it generates a feature vector for each patient, then aggregates them into a feature matrix (features are sorted alphabetically by name), and lastly performs imputation and normal- ization with respect to the population. After all these steps, it splits the data into 80% train and 20% test. **Report (i) the dimensionality $d$ of feature vector (ii) the average feature vector, and the corresponding name for each feature of the training set.**

In [0]:
X_train, y_train, X_test, y_test, feature_names = get_train_test_split()

In [0]:
print("Dimensionality:")
print(f"d = {X_train.shape[1]}")

In [0]:
print("Average feature vector:")
print(pd.DataFrame({"Feature Name": feature_names, "Mean value": X_train.mean(axis=0)}))

# 2 - Hyperparameter and model selection

## 2.1 - Linear-kernel SVM

### 2.1a

To begin, complete the implementation of the function cv\_performance(clf, X, y, metric=\textquotesingle accuracy\textquotesingle, k=5) as defined in the skeleton code. Here, you will make use of fit(X,y)  in the SVC class. The function returns the mean $k$-fold CV performance for the performance metric passed into the function. You should make use of the performance function in helper.py. When dividing the data into folds for CV, we use StratifiedKFold to try to keep the class proportions (ratio of positive to negative labels) roughly the same across folds. **Why might it be beneficial to maintain class proportions across folds?**

In [0]:
def cv_performance(clf, X, y, metric='auroc', k=5):
    """
    Splits the data X and the labels y into k-folds and runs k-fold
    cross-validation: for each fold i in 1...k, trains a classifier on
    all the data except the ith fold, and tests on the ith fold.
    Calculates the k-fold cross-validation performance metric for classifier
    clf by averaging the performance across folds.
    Input:
        clf: an instance of SVC()
        X: (n,d) array of feature vectors, where n is the number of examples
           and d is the number of features
        y: (n,) array of binary labels {1,-1}
        k: an int specifying the number of folds (default=5)
        metric: string specifying the performance metric (default='auroc'
             other options: 'f1-score', 'auroc', 'precision', 'sensitivity',
             and 'specificity')
    Returns:
        average 'validation' performance across the k folds as np.float64
    """
    # TODO: Finish implementing this function
    skf = StratifiedKFold(n_splits=k)
    scores = []
    # For each split in the k folds...
    for train, val in skf.split(X,y):
    # Fit the data to the training data...
        X_train, y_train, X_val, y_val = X[train], y[train], X[val], y[val]
        # Fit the data to the training data...
        clf.fit(X_train,y_train)
        # And test on the val fold.
        score = performance(clf, X_val, y_val, metric)
        scores.append(score)

    # And return the average performance across all fold splits.
    return np.array(scores).mean()

### 2.1b
Now implement the select param linear(X, y, metric='accuracy', k=5, C range=[]) function to choose a value of C for a linear SVM, using 5-fold cross validation on the training data maximizing AUROC. This function should call the cv performance function that you implemented in part(a), passing in instances of SVC(kernel='linear', C=C).


In [0]:
def select_param_linear(X, y, metric='auroc', k=5, C_range = []):
    """
    Sweeps different settings for the hyperparameter of a linear-kernel SVM,
    calculating the k-fold CV performance for each setting on X, y.
    Input:
        X: (n,d) array of feature vectors, where n is the number of examples
        and d is the number of features
        y: (n,) array of binary labels {1,-1}
        k: int specifying the number of folds (default=5)
        metric: string specifying the performance metric (default='accuracy',
             other options: 'f1-score', 'auroc', 'precision', 'sensitivity',
             and 'specificity')
        C_range: an array with C values to be searched over
    Returns:
        The parameter value for a linear-kernel SVM that maximizes the
        average 5-fold CV performance.
    """
    # TODO: Finish implementing this function
    #HINT: You should be using your cv_performance function here
    #to evaluate the performance of each SVM
    print("Linear SVM Hyperparameter Selection based on %s:" %metric)
    scores = []
    # Iterate over all of the given c parameters...
    for c in C_range:
        #Calculate the average performance on k-fold cross-validation
        clf = get_classifier(kernel='linear', C=c)
        score = cv_performance(clf, X, y, metric, k)
        print("C: %.3f score: %.4f" % (c, score))
        scores.append((c, score))
        
    # Return the C value with the maximum score
    maxval = max(scores, key=lambda x: x[1])
    return maxval[0]


### 2.1c

Finally, using the training data and the functions implemented above, find the best setting for C (searching over a range of values for C chosen in powers of 10 between $10^{-2}$ and $10^2$)

In [0]:
best_C = select_param_linear(X_train, y_train, 'auroc', 5, np.logspace(-2, 2, 5))

In [0]:
print(f"Performance against C increases then decreases. Best C is {best_C:.3f}")

### 2.1d
Now, using the value of C that maximizes your chosen performance measure, create an SVM as in the previous question. Train this SVM on the training data X_train, y_train. **Report the performance of this SVM on the test data X_test, y_test and plot the ROC curve (you may use the plot ROC curve function in helper.py)**

In [0]:
linear_clf = get_classifier(C=best_C)
linear_clf.fit(X_train, y_train)
test_perf = performance(linear_clf, X_test, y_test, 'auroc')
print(f"Test performance is {test_perf:.4f}")

In [0]:
plot_ROC_curve(linear_clf, X_test, y_test)

### 2.1e

Recall that each coefficient of \theta is associated with a clinical feature in your feature vector. The more positive a coefficient is, the more that feature contributes to a positive label. Similarly, the more negative a coefficient is, the more that feature contributes to a negative label. By examining these coefficients, we can start to generate hypotheses regarding what features are associated with increased (or decreased) risk of in-hospital mortality. Using C = 1.0, train an SVM on X train, y train. On this trained SVM, rank the features by their coefficients (hint: use coef [0] attribute of your classifier to obtain the learned coefficients. **Do these feature weights make sense?**

In [0]:
linear_clf = get_classifier(C=1)
linear_clf.fit(X_train, y_train)
weights = pd.DataFrame({'feature_name': feature_names, 'weight': linear_clf.coef_[0]})
print(f"Features ranked per coefficient:")
print(weights.sort_values(by='weight', ascending=False))

## 2.2 - RBF-kernel SVM

Now, we will perform hyperparameter selection for an RBF-kernel SVM. 

### 2.2a

Implement the select param rbf(X, y, metric='accuracy', k=5, param range=[]) function to choose a setting for C and " similar to the previous part. Your function should call your CV function (implemented earlier) passing in instances of SVC(kernel='rbf', C=C, gamma=gamma), with different values for C and ". Again, you may choose to use the helper function get classifier. Your function will perform a Grid Search over possible combinations of (C, ") and choose the best setting after tuning. The function argument param range is a two-column matrix, where the first column contains values of C and the second column contains values of ". Each row of this matrix is a distinct (C, ") pair to try.

In [0]:
def select_param_rbf(X, y, metric='auroc', k=5, param_range=[]):
    """
    Sweeps different settings for the hyperparameters of an RBF-kernel SVM,
    calculating the k-fold CV performance for each setting on X, y.
    Input:
        X: (n,d) array of feature vectors, where n is the number of examples
           and d is the number of features
        y: (n,) array of binary labels {1,-1}
        k: an int specifying the number of folds (default=5)
        metric: string specifying the performance metric (default='accuracy'
                 other options: 'f1-score', 'auroc', 'precision', 'sensitivity',
                 and 'specificity')
        parameter_values: a (num_param, 2)-sized array containing the
            parameter values to search over. The first column should
            represent the values for C, and the second column should
            represent the values for gamma. Each row of this array thus
            represents a pair of parameters to be tried together.
    Returns:
        The parameter value(s) for a RBF-kernel SVM that maximize
        the average 5-fold CV performance
    """
    # TODO: Finish implementing this function
    # Hint: This will be very similar to select_param_linear, except
    # the type of SVM model you are using will be different...
    print("RBF SVM Hyperparameter Selection based on %s:" %metric)
    scores = []
    # For each parameter pair to try...
    for p in param_range:
        c= p[0]
        gamma = p[1]
        clf = get_classifier(kernel='rbf', C=c, gamma=gamma)
        # Determine the performance of the defined SVM
        score = cv_performance(clf, X, y, metric, k)
        print("C: %.3f gamma: %.3f score: %.4f" % (c, gamma, score))
        scores.append((c, gamma, score))
    # And report the pair (C,gamma) that yielded the best metric performance
    maxval = max(scores, key=lambda x: x[2])
    return maxval[0], maxval[1]

### 2.2b

Using the training data the function implemented in (a), find the best values for C and " for AUROC using Grid Search and 5-fold cross validation. Search over powers of 10 for both C (between $10^{−2}$ and $10^2$) and \gamma (between $10^{−2}$ and $10^2$) [a total of 25 pairs]. **What setting maximizes cross-validation AUROC?** 

In [0]:
import itertools

In [0]:
C_range = np.logspace(-2, 2, 5)
coeff_range = list(np.logspace(-2, 2, 5))
param_range = list(itertools.product(C_range, coeff_range))
best_C, best_gamma = select_param_rbf(X_train, y_train, 'auroc', 5, param_range)

In [0]:
print(f"The best pair of parameters is (C, gamma) = ({best_C:.6f}, {best_gamma:.5f})")

Using this setting, train a classifier using all of the training data and report performance on the test set. **How does it’s performance compare to the linear classifier you trained earlier?**

In [0]:
rbf_clf = get_classifier(kernel='rbf', C=best_C, gamma=best_gamma)
rbf_clf.fit(X_train, y_train)
test_perf = performance(rbf_clf, X_test, y_test, "auroc")
print(f"Test performance is {test_perf:.4f}")

In [0]:
plot_ROC_curve(rbf_clf, X_test, y_test)